In [1]:
import numpy as np
import igl
import meshplot as mp



In [2]:
v, f = igl.read_triangle_mesh('./data/camel_head.off')
kv_min, kv_max, k_min, k_max = igl.principal_curvature(v, f)

In [3]:
def regularized():
    regular = []
    singular = []
    color = []
    for i, t in enumerate(f):
        if kv_max[t[0]] @ kv_max[t[1]] < 0:
            kv_max[t[1]] *= -1
        if kv_min[t[0]] @ kv_min[t[1]] < 0:
            kv_min[t[1]] *= -1
        if kv_max[t[0]] @ kv_max[t[2]] < 0:
            kv_max[t[2]] *= -1
        if kv_min[t[0]] @ kv_min[t[2]] < 0:
            kv_min[t[2]] *= -1

        if kv_max[t[1]] @ kv_max[t[2]] > 0 and kv_min[t[1]] @ kv_min[t[2]] > 0:
            regular.append(i)
            color.append([1, 1, 1])
        else:
            singular.append(i)
            color.append([0.8, 0.8, 0.8])

    return np.array(regular), np.array(singular), np.array(color)

regular, singular, color = regularized()

#  Compute discrete extremalities

In [4]:
def compute_extremalities():
    A = igl.doublearea(v, f)
    G = igl.grad(v, f)
    VF, NI = igl.vertex_triangle_adjacency(f, v.shape[0])
    g_max = np.reshape(G @ k_max, f.shape, order='F')
    g_min = np.reshape(G @ k_min, f.shape, order='F')
    e_max = np.zeros((v.shape[0], 1))
    e_min = np.zeros((v.shape[0], 1))
    
    for p in range(v.shape[0]):
        star = [VF[NI[p] + i] for i in range(NI[p + 1] - NI[p])]
        area = 0
        ei_max = 0
        ei_min = 0
        for t in star:
            area += A[t]
            ei_max += A[t] * (g_max[t] @ kv_max[p])
            ei_min += A[t] * (g_min[t] @ kv_min[p])
        e_max[p] = ei_max / area
        e_min[p] = ei_min / area
    return e_max, e_min

e_max, e_min = compute_extremalities()
# print(e_max)

# Smooth discrete extremalities

In [5]:
def smooth_extremalities(v, f, k, kd, ex):
    A = igl.adjacency_list(f)
    L = igl.cotmatrix(v,f)
#     M = igl.massmatrix(v,f)
#     L = M.T * L 
    LEx = np.zeros((v.shape[0],1))
    for indvi, vi in enumerate(v):
        lp = 0
        for vn in A[indvi]:
            sign = 1 if np.dot(kd[indvi],kd[vn]) > 0 else -1
            lp += L[indvi,vn] * (sign * ex[vn] - ex[indvi])
        LEx[indvi] = lp
    ex += 0.02 * LEx
    return ex
for i in range(10): # smooth for 10 rounds
    e_max = smooth_extremalities(v, f, k_max, kv_max, e_max)
    e_min = smooth_extremalities(v, f, k_min, kv_min, e_min)

# Extraction of Feature Lines

In [80]:
def extract_feature_lines():
    feature_lines_max = [[], []]
    feature_lines_min = [[], []]
    lines_max_index = [[], []]
    lines_min_index = [[], []]
    ridge_vertices_max = []
    ridge_vertices_min = []
    k_max_polyline = []
    k_min_polyline = []
    marked_edges = []
    # process regular triangles
    f_regular = np.array([f[i] for i in regular])
    # e_max
    num = 0
    for i, t in enumerate(f_regular):
        e_max_p = np.array([e_max[p] for p in t])
        kv_max_p = np.array([kv_max[p] for p in t])
        k_max_p = np.array([k_max[p] for p in t])
        k_min_p = np.array([k_min[p] for p in t])
        # choose k_max and e_max in T consistently
        if kv_max_p[0] @ kv_max_p[1] < 0:
            kv_max_p[1] *= -1
            e_max_p[1] *= -1
        if kv_max_p[0] @ kv_max_p[2] < 0:
            kv_max_p[2] *= -1
            e_max_p[2] *= -1
        # if the zero set of emax in T is empty
        if e_max_p[e_max_p.argmax()] <= 0.0 or e_max_p[e_max_p.argmin()] >= 0.0:
            continue 
        # Compute gradient of e
        v_regular = np.array([v[p] for p in t])
        G_regular = igl.grad(v_regular, np.array([[0, 1, 2]]))
        ge_max = G_regular * e_max_p

        # equation 5 & 6
        if not ((kv_max_p.sum(axis=0) @ ge_max) < 0 
            and abs(k_max_p.sum(axis=0)) - abs(k_min_p.sum(axis=0)) > 0):
            continue
        ridge_vertices = []
        
        index = 0
        for i in range(3):
            j = (i + 1) % 3
            if e_max_p[i] * e_max_p[j] > 0:
                continue
            num += 1

            a = abs(e_max_p[i])
            b = abs(e_max_p[j])
            p = (b * v[t[i]] + a * v[t[j]]) / (a + b)
            p_tuple = p.tobytes()
            if p_tuple not in ridge_vertices_max:
                ridge_vertices_max.append(p_tuple)
                k_max_polyline.append((b * k_max[t[i]] + a * k_max[t[j]]) / (a + b))
                lines_max_index[index].append(len(k_max_polyline) - 1)
            else:
                lines_max_index[index].append(ridge_vertices_max.index(p_tuple))
            ridge_vertices.append(p)
            if [t[i], t[j]] not in marked_edges:
                marked_edges.append([t[i], t[j]])
            index = 1 - index
        
        if len(ridge_vertices) == 2:
            feature_lines_max[0].append(ridge_vertices[0])
            feature_lines_max[1].append(ridge_vertices[1])

    # e_min
    for i, t in enumerate(f_regular):
        e_min_p = np.array([e_min[p] for p in t])
        kv_min_p = np.array([kv_min[p] for p in t])
        k_max_p = np.array([k_max[p] for p in t])
        k_min_p = np.array([k_min[p] for p in t])
        # choose k_max and e_max in T consistently
        if kv_min_p[0] @ kv_min_p[1] < 0:
                kv_min_p[1] *= -1
                e_min_p[1] *= -1
        if kv_min_p[0] @ kv_min_p[2] < 0:
                kv_min_p[2] *= -1
                e_min_p[2] *= -1

        # if the zero set of emax in T is empty
        if np.max(e_min_p) < 0 or np.min(e_min_p) > 0:
            continue
        # Compute gradient of e
        v_regular = np.array([v[p] for p in t])
        G_regular = igl.grad(v_regular, np.array([[0, 1, 2]]))
        ge_min = G_regular * e_min_p

        # equation 5 & 6
        if not ((kv_min_p.sum(axis=0) @ ge_min) > 0 
            and abs(k_max_p.sum(axis=0)) - abs(k_min_p.sum(axis=0)) < 0):
            continue

        ridge_vertices = []
        
        for i in range(3):
            j = (i + 1) % 3
            if e_min_p[i] * e_min_p[j] > 0:
                continue
            a = abs(e_min_p[i])
            b = abs(e_min_p[j])
            p = (b * v[t[i]] + a * v[t[j]]) / (a + b)
            p_tuple = p.tobytes() 
            if p_tuple not in ridge_vertices_min:
                ridge_vertices_min.append(p_tuple)
                k_min_polyline.append((b * k_min[t[i]] + a * k_min[t[j]]) / (a + b))
                lines_min_index[index].append(len(k_min_polyline) - 1)
            else:
                lines_min_index[index].append(ridge_vertices_min.index(p_tuple))
            ridge_vertices.append(p)
            if [t[i], t[j]] not in marked_edges:
                marked_edges.append([t[i], t[j]])
            index = 1 - index
        
        if len(ridge_vertices) == 2:
            feature_lines_min[0].append(ridge_vertices[0])
            feature_lines_min[1].append(ridge_vertices[1])

    # process singular triangles
    f_singular = np.array([f[i] for i in singular])
    # e_max
    for i, t in enumerate(f_singular):
        marked_edges_t = []
        for i in range(3):
            j = (i + 1) % 3
            if [t[i], t[j]] in marked_edges or [t[j], t[i]] in marked_edges:
                marked_edges_t.append([t[i], t[j]])
        
        if len(marked_edges_t) == 2:
            for i in range(2):
                v1 = marked_edges_t[i][0]
                v2 = marked_edges_t[i][1]
                a = abs(e_max[v1])
                b = abs(e_max[v2])
                p = (b * v[v1] + a * v[v2]) / (a + b)
                p_tuple = p.tobytes() 
                ridge_vertices_max=[]
                if p_tuple not in ridge_vertices_max:
                    ridge_vertices_max.append(p_tuple)
                    k_max_polyline.append((b * k_max[v1] + a * k_max[v2]) / (a + b))
                    lines_max_index[i].append(len(k_max_polyline) - 1)
                else:
                    lines_max_index[i].append(ridge_vertices_max.index(p))
                feature_lines_max[i].append(p)
        if len(marked_edges_t) == 3:
            v1 = v[f[t]][0]
            v2 = v[f[t]][1]
            v3 = v[f[t]][2]
            barycenter = (v1 + v2 + v3) / 3
            # print("Type of v1 before:", type(v1), "Value of v1:", v1)
            # print("Type of v2 before:", type(v2), "Value of v2:", v2)
            # print("Type of v3 before:", type(v3), "Value of v3:", v3)
            # print("Type of barycenter:", type(barycenter), "Value of a:", barycenter)
            for i in range(3):
                v1 = marked_edges_t[i][0]
                v2 = marked_edges_t[i][1]
                v3 = marked_edges_t[i][1]
                a = abs(e_max[v1])
                b = abs(e_max[v2])
                # Print statements to check types
                # print("Type of a:", type(a), "Value of a:", a)
                # print("Type of b:", type(b), "Value of b:", b)
                # print("Type of v1:", type(v1), "Value of v1:", v1)
                # print("Type of v2:", type(v2), "Value of v2:", v2)
                ridge_vertices_max=[]
                if [barycenter[0], barycenter[1], barycenter[2]] not in ridge_vertices_max:
                    ridge_vertices_max.append(barycenter)
                    k_max_polyline.append((k_max[v1] + k_max[v2] + k_max[v3]) / 3)
                    lines_max_index[0].append(len(k_max_polyline) - 1)
                else:
                    lines_max_index[0].append(ridge_vertices_max.index(barycenter))
                feature_lines_max[0].append(barycenter[0])
                feature_lines_max[0].append(barycenter[1])
                feature_lines_max[0].append(barycenter[2])
                p = (b * v[v1] + a * v[v2]) / (a + b)
                p_tuple = p.tobytes()
                ridge_vertices_max=[]
                # print("Computed p:", p)
                # print("Byte representation of p (p_tuple):", p_tuple)
                if p_tuple not in ridge_vertices_max:
                    ridge_vertices_max.append(p_tuple)
                    k_max_polyline.append((b * k_max[v1] + a * k_max[v2]) / (a + b))
                    lines_max_index[1].append(len(k_max_polyline) - 1)
                else:
                    lines_max_index[1].append(ridge_vertices_max.index(p_tuple))
                feature_lines_max[1].append(p)
    # e_min
    for i, t in enumerate(f_singular):
        marked_edges_t = []
        for i in range(3):
            j = (i + 1) % 3
            if [t[i], t[j]] in marked_edges or [t[j], t[i]] in marked_edges:
                marked_edges_t.append([t[i], t[j]])
        
        if len(marked_edges_t) == 2:
            for i in range(2):
                v1 = marked_edges_t[i][0]
                v2 = marked_edges_t[i][1]
                v3 = marked_edges_t[i][1]
                a = abs(e_min[v1])
                b = abs(e_min[v2])
                p = (b * v[v1] + a * v[v2]) / (a + b)
                p_tuple = p.tobytes() 
                if p_tuple not in ridge_vertices_min:
                    ridge_vertices_min.append(p_tuple)
                    k_min_polyline.append((b * k_min[v1] + a * k_min[v2]) / (a + b))
                    lines_min_index[i].append(len(k_min_polyline) - 1)
                else:
                    lines_min_index[i].append(ridge_vertices_min.index(p_tuple))
                feature_lines_min[i].append(p)
        if len(marked_edges_t) == 3:
            v1 = v[f[t]][0]
            v2 = v[f[t]][1]
            v3 = v[f[t]][2]
            barycenter = (v1 + v2 + v3) / 3
            for i in range(3):
                v1 = marked_edges_t[i][0]
                v2 = marked_edges_t[i][1]
                v3 = marked_edges_t[i][1]
                a = abs(e_min[v1])
                b = abs(e_min[v2])
                ridge_vertices_min=[]
                if [barycenter[0], barycenter[1], barycenter[2]] not in ridge_vertices_min:
                    ridge_vertices_min.append(barycenter)
                    k_min_polyline.append((k_min[v1] + k_min[v2] + k_min[v3]) / 3)
                    lines_min_index[0].append(len(k_min_polyline) - 1)
                else:
                    lines_min_index[0].append(ridge_vertices_min.index([barycenter[0], barycenter[1], barycenter[2]]))
                feature_lines_min[0].append(barycenter)
                p = (b * v[v1] + a * v[v2]) / (a + b)
                p_tuple = p.tobytes() 
                ridge_vertices_min=[]
                if p_tuple not in ridge_vertices_min:
                    ridge_vertices_min.append(p_tuple)
                    k_min_polyline.append((b * k_min[v1] + a * k_min[v2]) / (a + b))
                    lines_min_index[1].append(len(k_min_polyline) - 1)
                else:
                    lines_min_index[1].append(ridge_vertices_min.index(p_tuple))
                feature_lines_min[1].append(p)
                
    # print(feature_lines_min)
    return feature_lines_max, feature_lines_min, k_max_polyline, k_min_polyline, ridge_vertices_max, ridge_vertices_min, lines_max_index, lines_min_index

feature_lines_max, feature_lines_min, k_max_polyline, k_min_polyline, ridge_vertices_max, ridge_vertices_min, lines_max_index, lines_min_index = extract_feature_lines()

print(lines_max_index[1])


[1, 2, 4, 6, 7, 9, 11, 13, 15, 17, 19, 21, 23, 25, 27, 26, 30, 32, 34, 36, 38, 40, 42, 44, 46, 48, 49, 51, 53, 55, 56, 58, 60, 61, 63, 65, 67, 69, 71, 73, 74, 76, 78, 80, 82, 84, 86, 87, 89, 91, 37, 94, 96, 98, 100, 102, 3, 104, 106, 108, 110, 112, 114, 44, 8, 116, 10, 118, 120, 122, 12, 14, 124, 126, 127, 65, 16, 130, 74, 76, 22, 24, 135, 27, 137, 80, 139, 141, 142, 29, 143, 144, 138, 132, 146, 31, 34, 148, 149, 101, 38, 153, 155, 42, 157, 108, 160, 161, 109, 111, 163, 9, 43, 165, 167, 168, 169, 45, 49, 50, 173, 52, 175, 177, 56, 119, 180, 182, 184, 186, 184, 188, 183, 190, 61, 124, 127, 63, 193, 64, 196, 198, 199, 67, 201, 202, 203, 203, 206, 208, 210, 212, 130, 11, 19, 215, 216, 217, 218, 219, 220, 221, 72, 75, 224, 225, 121, 25, 227, 135, 230, 137, 232, 28, 234, 235, 236, 237, 82, 239, 240, 241, 242, 243, 245, 2, 246, 247, 248, 140, 249, 251, 89, 251, 253, 250, 255, 90, 256, 257, 259, 92, 261, 262, 263, 264, 180, 266, 96, 98, 268, 270, 272, 274, 276, 278, 280, 282, 284, 286, 288, 2

# Remove small ridges by a threshold filter

In [ ]:
def remove_small_ridges():
    T = 0.8
    feature_lines_max_large = [[], []]
    visited = np.zeros(len(feature_lines_max[0]))
    while 0 in visited:
        trapezoid = 0
        temp_lines = [[],[]]
        if(np.where(visited==0)[0][0] < len(lines_max_index[0])):
            index = np.where(visited==0)[0][0]
        else:
            index = len(lines_max_index[0]) - 1
        which_end = 0
        v1 = lines_max_index[0][index]
        v2 = lines_max_index[1][index]
        visited[index] = 1
        temp_lines[0].append(feature_lines_max[0][index])
        temp_lines[1].append(feature_lines_max[1][index])
        trapezoid += 0.5 * (k_max_polyline[v1] + k_max_polyline[v2]) * np.linalg.norm(feature_lines_max[0][index] - feature_lines_max[1][index])
        stack = [v1, v2]
        while stack:
            v1 = stack.pop(0)
            for i in np.where(np.array(lines_max_index[0]) == v1)[0]:
                if (visited[i]):
                    continue
                a = lines_max_index[0][i]
                b = lines_max_index[1][i]
                visited[i] = 1
                temp_lines[0].append(feature_lines_max[0][i])
                temp_lines[1].append(feature_lines_max[1][i])
                trapezoid += 0.5 * (k_max_polyline[a] + k_max_polyline[b]) * np.linalg.norm(feature_lines_max[0][i] - feature_lines_max[1][i])
                stack.append(a)
                stack.append(b)
            for i in np.where(np.array(lines_max_index[1]) == v1)[0]:
                if (visited[i]):
                    continue
                a = lines_max_index[0][i]
                b = lines_max_index[1][i]
                visited[i] = 1
                temp_lines[0].append(feature_lines_max[0][i])
                temp_lines[1].append(feature_lines_max[1][i])
                trapezoid += 0.5 * (k_max_polyline[a] + k_max_polyline[b]) * np.linalg.norm(feature_lines_max[0][i] - feature_lines_max[1][i])
                stack.append(a)
                stack.append(b)
            v2 = stack.pop(0)
            for i in np.where(np.array(lines_max_index[0]) == v2)[0]:
                if (visited[i]):
                    continue
                a = lines_max_index[0][i]
                b = lines_max_index[1][i]
                visited[i] = 1
                temp_lines[0].append(feature_lines_max[0][i])
                temp_lines[1].append(feature_lines_max[1][i])
                trapezoid += 0.5 * (k_max_polyline[a] + k_max_polyline[b]) * np.linalg.norm(feature_lines_max[0][i] - feature_lines_max[1][i])
                stack.append(a)
                stack.append(b)
            for i in np.where(np.array(lines_max_index[1]) == v2)[0]:
                if (visited[i]):
                    continue
                a = lines_max_index[0][i]
                b = lines_max_index[1][i]
                visited[i] = 1
                temp_lines[0].append(feature_lines_max[0][i])
                temp_lines[1].append(feature_lines_max[1][i])
                trapezoid += 0.5 * (k_max_polyline[a] + k_max_polyline[b]) * np.linalg.norm(feature_lines_max[0][i] - feature_lines_max[1][i])
                stack.append(a)
                stack.append(b)
        if trapezoid > T:
            for i, _ in enumerate(temp_lines[0]):
                feature_lines_max_large[0].append(temp_lines[0][i])
                feature_lines_max_large[1].append(temp_lines[1][i])

    feature_lines_min_large = [[], []]
    visited = np.zeros(len(feature_lines_min[0]))
    while 0 in visited:
        trapezoid = 0
        temp_lines = [[],[]]
        index = np.where(visited==0)[0][0]
        which_end = 0
        v1 = lines_min_index[0][index]
        v2 = lines_min_index[1][index]
        visited[index] = 1
        temp_lines[0].append(feature_lines_min[0][index])
        temp_lines[1].append(feature_lines_min[1][index])
        trapezoid += 0.5 * (k_min_polyline[v1] + k_min_polyline[v2]) * np.linalg.norm(feature_lines_min[0][index] - feature_lines_min[1][index])
        stack = [v1, v2]
        while stack:
            v1 = stack.pop(0)
            for i in np.where(np.array(lines_min_index[0]) == v1)[0]:
                if (visited[i]):
                    continue
                a = lines_min_index[0][i]
                b = lines_min_index[1][i]
                visited[i] = 1
                temp_lines[0].append(feature_lines_min[0][i])
                temp_lines[1].append(feature_lines_min[1][i])
                trapezoid += 0.5 * (k_min_polyline[a] + k_min_polyline[b]) * np.linalg.norm(feature_lines_min[0][i] - feature_lines_min[1][i])
                stack.append(a)
                stack.append(b)
            for i in np.where(np.array(lines_min_index[1]) == v1)[0]:
                if (visited[i]):
                    continue
                a = lines_min_index[0][i]
                b = lines_min_index[1][i]
                visited[i] = 1
                temp_lines[0].append(feature_lines_min[0][i])
                temp_lines[1].append(feature_lines_min[1][i])
                trapezoid += 0.5 * (k_min_polyline[a] + k_min_polyline[b]) * np.linalg.norm(feature_lines_min[0][i] - feature_lines_min[1][i])
                stack.append(a)
                stack.append(b)
            v2 = stack.pop(0)
            for i in np.where(np.array(lines_min_index[0]) == v2)[0]:
                if (visited[i]):
                    continue
                a = lines_min_index[0][i]
                b = lines_min_index[1][i]
                visited[i] = 1
                temp_lines[0].append(feature_lines_min[0][i])
                temp_lines[1].append(feature_lines_min[1][i])
                trapezoid += 0.5 * (k_min_polyline[a] + k_min_polyline[b]) * np.linalg.norm(feature_lines_min[0][i] - feature_lines_min[1][i])
                stack.append(a)
                stack.append(b)
            for i in np.where(np.array(lines_min_index[1]) == v2)[0]:
                if (visited[i]):
                    continue
                a = lines_min_index[0][i]
                b = lines_min_index[1][i]
                visited[i] = 1
                temp_lines[0].append(feature_lines_min[0][i])
                temp_lines[1].append(feature_lines_min[1][i])
                trapezoid += 0.5 * (k_min_polyline[a] + k_min_polyline[b]) * np.linalg.norm(feature_lines_min[0][i] - feature_lines_min[1][i])
                stack.append(a)
                stack.append(b)
        if -trapezoid > T:
            for i, _ in enumerate(temp_lines[0]):
                feature_lines_min_large[0].append(temp_lines[0][i])
                feature_lines_min_large[1].append(temp_lines[1][i])

    return np.array(feature_lines_max_large), np.array(feature_lines_min_large)
feature_lines_max_large, feature_lines_min_large = remove_small_ridges()

In [ ]:

p = mp.plot(v, f, c=color)
p.add_lines(feature_lines_max_large[0], feature_lines_max_large[1], shading={"line_color": "red"})
p.add_lines(feature_lines_min_large[0], feature_lines_min_large[1], shading={"line_color": "blue"})